In [ ]:
from google.cloud import vision
import os
import io
import json

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/tak/Desktop/temp/tak-playground-d69a7d8fcb5c.json"
client = vision.ImageAnnotatorClient()

image_path = '/Users/tak/Desktop/temp/WhatsApp Image 2021-01-22 at 1.45.36 PM.jpeg'
with io.open(image_path, 'rb') as image_file:
    content = image_file.read()
image = vision.types.Image(content=content)
response = client.text_detection(
    image=image)  # returns TextAnnotation
texts = response.text_annotations

    

In [ ]:
# texts[2].description
texts[0].description.split('\n')

In [ ]:
%%time
import requests as req
from lxml import etree
#from bs4 import BeautifulSoup
resp = req.get("https://hk.finance.yahoo.com/quote/9733.HK/history/")
#soup = BeautifulSoup(resp.text, 'lxml')
content = resp.content.decode()

In [ ]:
import pandas as pd
## parameters
margin = 0.5

df = pd.DataFrame(columns =["name","date","start","h_price","l_price","f_price","real_f_price","volume","margin"])

html = etree.HTML(content)
stock_id='0001'

In [ ]:
import pandas as pd
import requests as req
from lxml import etree
from multiprocessing.pool import ThreadPool
from functools import partial

class S_W_old():
    def __init__(self,l,margin):
        self.list= l
        self.margin = margin
        self.df = pd.DataFrame(columns =["name","date","start","h_price","l_price","f_price","real_f_price","volume","margin"])
    
    def scrape(self,stock_id):
    # for stock_id in self.list:
        resp = req.get(f"https://hk.finance.yahoo.com/quote/{stock_id}.HK/history/")
        content = resp.content.decode()
        html = etree.HTML(content)
        num = len(html.xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/*/td[1]/span/text()'))
        if num != 0:
            # for i in range(1,num+1):
            for i in range(2,4):
                to_append=[]
                for j in range(9):
                    if j == 0:
                        to_append.append(stock_id)
                    elif j == 7:
                        to_append.append(html.xpath(f'/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr[{i}]/td[{j}]/span/text()')[0].replace(',',''))
                    elif j == 8:
                        to_append.append('N')
                    else:
                        try:
                            to_append.append(html.xpath(f'/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr[{i}]/td[{j}]/span/text()')[0])
                        except:
                                to_append.append(0)
                a_series = pd.Series(to_append, index = df.columns)
                self.df = self.df.append(a_series, ignore_index=True)
            ##change type and check margin
            self.df[["start","h_price","l_price","f_price","real_f_price","volume"]]=self.df[["start","h_price","l_price","f_price","real_f_price","volume"]].apply(pd.to_numeric)
            if (self.df[self.df['name'] == stock_id]['volume'].iloc[0] - self.df[self.df['name'] == stock_id]['volume'].iloc[1])/self.df[self.df['name'] == stock_id]['volume'].iloc[1] > self.margin:
                self.df.loc[self.df['name'] == stock_id,"margin"] ='Y'
    # return self.df
    def main(self):
        for i in self.list:
            self.scrape(i)
        return self.df

In [ ]:
#df[df['name'] == '2126'].groupby('name')['volume'].pct_change()
df

In [ ]:
# df[df['name'] == stock_id]["margin"]=df[df['name'] == stock_id]["margin"].apply(lambda x : x.replace('N','Y'))
#df.loc[df['name'] == stock_id,"margin"] ='Y'

In [ ]:
df

In [7]:
import pandas as pd
import requests as req
from lxml import etree
from multiprocessing.pool import ThreadPool
from functools import partial

class S_W():
    def __init__(self,l,margin):
        self.list= l
        self.margin = margin
        self.df = pd.DataFrame(columns =["name","date","start","h_price","l_price","f_price","real_f_price","volume","margin"])
    
    def scrape(self,stock_id):
    # for stock_id in self.list:
        resp = req.get(f"https://hk.finance.yahoo.com/quote/{stock_id}.HK/history/")
        content = resp.content.decode()
        html = etree.HTML(content)
        num = len(html.xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/*/td[1]/span/text()'))
        if num != 0:
            # for i in range(1,num+1):
            for i in range(1,3):
                to_append=[]
                for j in range(9):
                    if j == 0:
                        to_append.append(stock_id)
                    elif j == 7:
                        try:
                            to_append.append(html.xpath(f'/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr[{i}]/td[{j}]/span/text()')[0].replace(',',''))
                        except:
                            to_append.append(0)
                    elif j == 8:
                        to_append.append('N')
                    else:
                        try:
                            to_append.append(html.xpath(f'/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr[{i}]/td[{j}]/span/text()')[0].replace(',',''))
                        except:
                                to_append.append(0)
                a_series = pd.Series(to_append, index = self.df.columns)
                self.df = self.df.append(a_series, ignore_index=True)
            ##change type and check margin
            self.df[["start","h_price","l_price","f_price","real_f_price","volume"]]=self.df[["start","h_price","l_price","f_price","real_f_price","volume"]].apply(pd.to_numeric)
            if (self.df[self.df['name'] == stock_id]['volume'].iloc[0] - self.df[self.df['name'] == stock_id]['volume'].iloc[1])/self.df[self.df['name'] == stock_id]['volume'].iloc[1] > self.margin:
                self.df.loc[self.df['name'] == stock_id,"margin"] ='Y'
    # return self.df
    def main(self):
        with ThreadPool(20) as pool:
            pool.map(self.scrape,self.list)
            pool.close()
            pool.join()
        return self.df




In [8]:
%%time
Test = S_W_old(['2158','2126','6999','6969','0369','9988','1093','6969','0388','6618'],2)
D = Test.main()
D
##1min17s


NameError: name 'S_W_old' is not defined

In [10]:
D[D["margin"] == 'Y']

,name,date,start,h_price,l_price,f_price,real_f_price,volume,margin
4,9900,2021年1月27日,1.420,1.420,1.390,1.390,1.390,80000,Y
5,9900,2021年1月26日,1.480,1.480,1.480,1.480,1.480,0,Y
32,9919,2021年1月28日,0.850,0.880,0.850,0.870,0.870,576000,Y
33,9919,2021年1月27日,0.870,0.870,0.860,0.870,0.870,114000,Y
46,9977,2021年1月28日,3.230,3.240,2.980,3.030,3.030,6401000,Y
47,9977,2021年1月27日,3.370,3.400,3.170,3.280,3.280,1895000,Y
60,9998,2021年1月27日,0.235,0.235,0.210,0.223,0.223,1175000,Y
61,9998,2021年1月26日,0.211,0.228,0.211,0.228,0.228,55000,Y
68,9983,2021年1月28日,7.680,7.960,7.490,7.550,7.550,6905000,Y
69,9983,2021年1月27日,7.580,7.580,7.150,7.400,7.400,2263000,Y


In [9]:
%%time
# Test = S_W(['2158','2126','6999','6969','0369','9988','1093','6969','0388','6618'],0.5)
# D = Test.main()
# D
##12.8s 1min 34s 26.7 s
temp_list = [str(i) for i in range(9900,10000)]
#temp_list = ['2338']
Test = S_W(temp_list,2)
D = Test.main()
D

CPU times: user 6.33 s, sys: 593 ms, total: 6.92 s
Wall time: 59.5 s


,name,date,start,h_price,l_price,f_price,real_f_price,volume,margin
0,9922,2021年1月28日,27.000,28.150,26.300,27.000,27.000,13200401,N
1,9922,2021年1月27日,28.800,29.200,26.600,27.450,27.450,10728081,N
2,9938,2021年1月28日,0.125,0.125,0.118,0.121,0.121,255000,N
3,9938,2021年1月27日,0.122,0.122,0.113,0.120,0.120,2820000,N
4,9900,2021年1月27日,1.420,1.420,1.390,1.390,1.390,80000,Y
...,...,...,...,...,...,...,...,...,...
77,9993,2021年1月27日,4.380,4.480,4.300,4.480,4.480,2591000,N
78,9991,2021年1月28日,112.000,113.200,108.000,108.200,108.200,376062,N
79,9991,2021年1月27日,105.800,125.000,105.800,120.000,120.000,3343345,N
80,9999,2021年1月28日,184.300,184.600,177.300,178.800,178.800,5639066,N
